In [1]:
import torch
from nltk.tokenize import word_tokenize
from torch.nn.utils.rnn import pad_sequence
import math
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Example text data
text_data = [
    "I love this movie!", "This movie is terrible.", "The acting was amazing.",
    "I hate this product.", "The food was delicious.", "I'm not satisfied with the service.",
    "This book is fantastic!", "The customer support was awful.", "I enjoyed my vacation.",
    "The quality of the product is poor.", "The movie made me cry.", "I feel great after watching it!",
    "The product exceeded my expectations.", "I wouldn't recommend it to anyone.",
    "I'm impressed with the performance.", "The experience was disappointing.",
    "The service was exceptional!", "I regret buying this.", "Best movie I've ever seen!",
    "This was a waste of money.", "I'm thrilled with my purchase.", "The worst experience ever.",
    "Absolutely amazing!", "A total disappointment.", "Highly recommended!",
    "Couldn't be more dissatisfied.", "So much fun!", "Complete waste of time.",
    "Brilliant performance!", "An utter disaster."
]

# Corresponding sentiment labels (1 for positive sentiment, 0 for negative sentiment)
sentiment_labels = torch.tensor(
    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0,
     1, 0, 1, 0, 1, 0, 1, 0, 1, 0], dtype=torch.float32)

# Tokenize and build vocabulary
tokenized_data = [word_tokenize(sentence.lower()) for sentence in text_data]
vocab = {word: idx for idx, word in enumerate(set(word for sentence in tokenized_data for word in sentence))}
index_data = [[vocab[word] for word in sentence] for sentence in tokenized_data]

# Convert to tensors and pad sequences
tensor_data = [torch.tensor(sentence) for sentence in index_data]
padded_data = pad_sequence(tensor_data, batch_first=True, padding_value=0)
max_length = padded_data.size(1)

class MultiheadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiheadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        self.head_dim = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_out = nn.Linear(d_model, d_model)

    def split_heads(self, x):
        batch_size, seq_length, _ = x.size()
        x = x.view(batch_size, seq_length, self.num_heads, self.head_dim)
        return x.permute(0, 2, 1, 3)

    def combine_heads(self, x):
        batch_size, num_heads, seq_length, _ = x.size()
        x = x.permute(0, 2, 1, 3).contiguous()
        return x.view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V):
        Q_transformed = self.W_q(Q)
        K_transformed = self.W_k(K)
        V_transformed = self.W_v(V)

        Q_split = self.split_heads(Q_transformed)
        K_split = self.split_heads(K_transformed)
        V_split = self.split_heads(V_transformed)

        attention_scores = torch.matmul(Q_split, K_split.transpose(-2, -1)) / math.sqrt(self.head_dim)
        attention_weights = F.softmax(attention_scores, dim=-1)
        attention_output = torch.matmul(attention_weights, V_split)

        attention_output = self.combine_heads(attention_output)
        output = self.W_out(attention_output)
        return output

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_length):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_length, d_model)
        position = torch.arange(0, max_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.attention = MultiheadAttention(d_model, num_heads)
        self.feedforwardnn = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, src_mask=None):
        attention_output = self.attention(x, x, x)
        x = self.norm1(x + self.dropout(attention_output))
        ff_output = self.feedforwardnn(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

class Encoder(nn.Module):
    def __init__(self, input_dim, d_model, num_heads, d_ff, num_layers, dropout, max_length):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_length)
        self.layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask=None):
        src = self.embedding(src)
        src = self.pos_encoding(src)
        src = self.dropout(src)

        for layer in self.layers:
            src = layer(src, src_mask)
        
        return src

class SentimentClassifier(nn.Module):
    def __init__(self, encoder, d_model, num_classes=1):
        super(SentimentClassifier, self).__init__()
        self.encoder = encoder
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, src, src_mask=None):
        encoder_output = self.encoder(src, src_mask)
        pooled_output = encoder_output.mean(dim=1)  # Global average pooling
        output = self.fc(pooled_output)
        return output

# Model hyperparameters
INPUT_DIM = len(vocab)
EMBEDDING_DIM = 256
NUM_HEADS = 8
HIDDEN_DIM = 512
NUM_LAYERS = 6
DROPOUT = 0.1
MAX_LENGTH = max_length

encoder = Encoder(INPUT_DIM, EMBEDDING_DIM, NUM_HEADS, HIDDEN_DIM, NUM_LAYERS, DROPOUT, MAX_LENGTH)
model = SentimentClassifier(encoder, EMBEDDING_DIM)

# Specify optimizer and loss function
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

# Create dataset and dataloader
dataset = TensorDataset(padded_data, sentiment_labels)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    for batch in iterator:
        optimizer.zero_grad()
        text, labels = batch
        text, labels = text.to(device), labels.to(device)
        predictions = model(text).squeeze(1)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

# Evaluation loop
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in iterator:
            text, labels = batch
            text, labels = text.to(device), labels.to(device)
            predictions = model(text).squeeze(1)
            loss = criterion(predictions, labels)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

# Training
num_epochs = 100
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}')

Epoch [1/100], Train Loss: 0.7267
Epoch [2/100], Train Loss: 2.7723
Epoch [3/100], Train Loss: 0.5921
Epoch [4/100], Train Loss: 0.5711
Epoch [5/100], Train Loss: 0.5445
Epoch [6/100], Train Loss: 0.4359
Epoch [7/100], Train Loss: 0.2956
Epoch [8/100], Train Loss: 0.1414
Epoch [9/100], Train Loss: 0.3297
Epoch [10/100], Train Loss: 0.4926
Epoch [11/100], Train Loss: 0.4521
Epoch [12/100], Train Loss: 0.0452
Epoch [13/100], Train Loss: 0.1285
Epoch [14/100], Train Loss: 0.2135
Epoch [15/100], Train Loss: 0.0688
Epoch [16/100], Train Loss: 0.0195
Epoch [17/100], Train Loss: 0.0202
Epoch [18/100], Train Loss: 0.0235
Epoch [19/100], Train Loss: 0.0314
Epoch [20/100], Train Loss: 0.0222
Epoch [21/100], Train Loss: 0.0158
Epoch [22/100], Train Loss: 0.0110
Epoch [23/100], Train Loss: 0.0092
Epoch [24/100], Train Loss: 0.0073
Epoch [25/100], Train Loss: 0.0065
Epoch [26/100], Train Loss: 0.0056
Epoch [27/100], Train Loss: 0.0048
Epoch [28/100], Train Loss: 0.0045
Epoch [29/100], Train Loss: 0

In [ ]:
import torch
from nltk.tokenize import word_tokenize

# Sample text for testing
test_text = "I not like this car"

# Preprocess the input text
def preprocess_text(text, vocab, max_length):
    tokenized = word_tokenize(text.lower())
    indexed = [vocab.get(word, 0) for word in tokenized]  # Use 0 for unknown words
    tensor = torch.tensor(indexed, dtype=torch.long)
    padded = torch.nn.functional.pad(tensor, (0, max_length - len(tensor)), value=0)  # Pad to max_length
    return padded

# Get the vocabulary size
vocab_size = len(vocab)

# Preprocess the test text
test_tensor = preprocess_text(test_text, vocab, max_length).unsqueeze(0)  # Add batch dimension

# Move the tensor to the appropriate device
test_tensor = test_tensor.to(device)

# Set the model to evaluation mode
model.eval()

# Perform the prediction
with torch.no_grad():
    output = model(test_tensor)
    prediction = torch.sigmoid(output).item()  # Apply sigmoid to get a probability

# Print the prediction
if prediction >= 0.5:
    print("Positive sentiment")
else:
    print("Negative sentiment")
